In [ ]:
import ipywidgets as wdg
from aitw_aiida_viscosity.cli.workflows.viscosity import launch_workflow

from common import VISCOSITY_WORKCHAIN_DEFAULTS, VISCOSITY_CODES, VISCOSITY_CODE_MAP

import utilities
import utilities_aiida as ua

In [ ]:
create_codes_button = wdg.Button(description='Create Codes')
output_area = wdg.Output()
def on_create_codes_button_clicked(b):
    with output_area:
        output_area.clear_output()
        print("Ensure required computer/codes are set up...")
        
        profile = ua.setup_profile()
        computer = ua.setup_computer()
        codes = ua.setup_codes(code_infos=VISCOSITY_CODES)

        print("Using profile:")
        print(f" - {profile}")
        print("Using computer:")
        print(f" - {computer}")
        print("Using codes: ")
        for code in codes:
            print(f" - {code}")

create_codes_button.on_click(on_create_codes_button_clicked)
display(create_codes_button, output_area)

Button(description='Create Codes', style=ButtonStyle())

Output()

In [ ]:
widgets = utilities.get_widgets_from_click_function(
    launch_workflow,
    override_defaults=VISCOSITY_WORKCHAIN_DEFAULTS,
    code_map=VISCOSITY_CODE_MAP,
)

title = wdg.HTML(value="<h2>AITW - AiiDA Monomer Viscosity Workchain</h2>")
display(title)
widget_box = wdg.VBox(list(widgets.values()))
display(widget_box)

generate_button = wdg.Button(description="Launch Workchain", button_style="success")
output_area = wdg.Output()

def on_generate_button_clicked(b):
    with output_area:
        output_area.clear_output()
        args = utilities.get_click_args_from_widgets(launch_workflow, widgets)
        launch_workflow(args, standalone_mode=False)

generate_button.on_click(on_generate_button_clicked)
display(generate_button)
display(output_area)

HTML(value='<h2>AITW - AiiDA Monomer Viscosity Workchain</h2>')

Button(button_style='success', description='Launch Workchain', style=ButtonStyle())

Output()

In [ ]:
import warnings
from IPython.display import Image

from aiida import orm

from aitw_aiida_viscosity.cli.data.viscosity_array import plot_viscosity

warnings.filterwarnings("ignore")

qb = orm.QueryBuilder()
qb.append(orm.ArrayData, filters={'attributes': {'has_key': 'array|deformation_velocities'}})
choices = [(str(_[0]), _[0].pk) for _ in qb.all()]

widgets = []
widgets.append(
    wdg.Dropdown(
        options=choices,
        description='Select Viscosity Data:',
        disabled=False,
        style={'description_width': '150px'},
    )
)

data_box = wdg.VBox(widgets)
display(data_box)
plot_button = wdg.Button(description="Plot Viscosity Data", button_style="info")

output_area = wdg.Output()

def on_plot_button_clicked(b):
    with output_area:
        output_area.clear_output()
        selected_data = widgets[0].value
        pk = selected_data
        filename = f"viscosity_data_{pk}.png"
        try:
            plot_viscosity([str(pk), '-o', filename], standalone_mode=False)
            pass
        except Exception as e:
            print(f"Warning: {e}")
        display(Image(filename=filename))
    

plot_button.on_click(on_plot_button_clicked)
display(plot_button)
display(output_area)

[('uuid: 79038142-0f3a-4823-816b-0f76a92c80de (pk: 290)', 290), ('uuid: 0749f785-7cd4-42f7-8e41-3347e69efaf6 (pk: 579)', 579)]


Button(button_style='info', description='Plot Viscosity Data', style=ButtonStyle())

Output()